# COVID 19: Forecasting using ARIMA and LSTM

In [1]:
import pandas as pd
import numpy as np

%load_ext nb_black

<IPython.core.display.Javascript object>

## Data Sources

In [2]:
confirmed_df = pd.read_csv("data/confirmed_df.csv")
deaths_df = pd.read_csv("data/deaths_df.csv")
recovered_df = pd.read_csv("data/recovered_df.csv")

<IPython.core.display.Javascript object>

In [3]:
# Dropping Serial Number
confirmed_df.drop(["SNo"], axis=1, inplace=True)
deaths_df.drop(["SNo"], axis=1, inplace=True)
recovered_df.drop(["SNo"], axis=1, inplace=True)

<IPython.core.display.Javascript object>

In [4]:
confirmed_df.head()

,Date,Province/State,Country,Confirmed
0,01/22/2020,Hong Kong,Hong Kong,0.0
1,01/22/2020,Macau,Macau,1.0
2,01/22/2020,Anhui,Mainland China,1.0
3,01/22/2020,Beijing,Mainland China,14.0
4,01/22/2020,Chongqing,Mainland China,6.0


<IPython.core.display.Javascript object>

In [5]:
deaths_df.head()

,Date,Province/State,Country,Deaths
0,01/22/2020,Hong Kong,Hong Kong,0.0
1,01/22/2020,Macau,Macau,0.0
2,01/22/2020,Anhui,Mainland China,0.0
3,01/22/2020,Beijing,Mainland China,0.0
4,01/22/2020,Chongqing,Mainland China,0.0


<IPython.core.display.Javascript object>

In [6]:
recovered_df.head()

,Date,Province/State,Country,Recovered
0,01/22/2020,Hong Kong,Hong Kong,0.0
1,01/22/2020,Macau,Macau,0.0
2,01/22/2020,Anhui,Mainland China,0.0
3,01/22/2020,Beijing,Mainland China,0.0
4,01/22/2020,Chongqing,Mainland China,0.0


<IPython.core.display.Javascript object>

## LSTM